# Estimating the Model Evidence using Bridge Sampling

## Background

In the previous tutorial we showed how one can esstimate the **model evidence** $\mathcal{Z}$ for two models in order to perform **Bayesian model comparison**. To this end, we computed the **Bayes factor** as the ratio of the two model evidences (i.e. one for each model). The estimates of $\mathcal{Z}$ which we used were the ones provided by ``pocoMC`` at the end of the *PMC* run.

**There is however another way of estimating the model evidence $\mathcal{Z}$, called Bridge Sampling, that is much more accurate.**

Bridge sampling estimates the model evidence $\mathcal{Z}$ using an auxiliary distribution $\mathcal{Q}(\theta)$, in addition to the posterior distribution $\mathcal{P}(\theta)$. The basic idea is to use samples from both the auxiliary and the posterior distribution in order to get the evidence:

$$
\mathcal{Z}_{\mathcal{P}} = \frac{\mathbb{E}_{\mathcal{Q}}[\alpha(\theta)\mathcal{P}(\theta)]}{\mathbb{E}_{\mathcal{P}}[\alpha(\theta)\mathcal{Q}(\theta)]}\mathcal{Z}_{\mathcal{Q}}
$$

where $\alpha(\theta)$ is an arbitrary function, called *bridge function, which controls the efficiency of the estimator. The use of samples from both distributions is characteristic of Bridge Sampling, unlike other estimators (e.g. Importance Sampling, Harmonic Mean, Nested Sampling, etc.) which rely on a single distribution. This important difference allows Bridge Sampling to constrain $\mathcal{Z}$ both from above and below thus resulting in very accurate and unbiased estimates.

In ``pocoMC`` we implement the powerful [**Gaussianised Bridge Sampling**](https://arxiv.org/abs/1912.06073) algorithm which utilises the *Normalising Flow (NF)* as the auxiliary distribution. Furthermore, the bridge function $\alpha(\theta)$ has been chosen such that the estimate of $\mathcal{Z}$ is optimal (i.e. lowest variance).

## Defining the likelihood and prior

For this example we will use a simple 2-D Rosenbrock (banana) distribution with a uniform prior $\mathcal{U}(-5,5)$. The reason for this is that we know the analytical value of $\log\mathcal{Z}_{an} = -5.804$. Of course the method of Bridge Sampling can be applied to any posterior distribution with a proper prior.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pocomc as pc
np.random.seed(42)

n_dim = 2

low = -5.0
high = 5.0

lower = np.full(n_dim, low) # lower bound of the prior
upper = np.full(n_dim, high) # upper bound of the prior
bound = np.array((lower, upper)).T
diff = bound[:, 1] - bound[:, 0]
const = np.sum(np.log(diff))

def log_prior(x):
    if np.any(x>high) or np.any(x<low):
        return -np.inf 
    else:
        return -const

# vectorised loglikelihood
def log_like_vec(x):

    f = np.sum(100.0*(x[:,::2]**2.0 - x[:,1::2])**2.0 + (x[:,::2] - 1.0)**2.0, axis=1)

    return - f

## Running PMC

The next step is to actually sample from the posterior using PMC as usual.

In [2]:
n_particles = 1000

prior_samples = np.random.uniform(low, high, size=(n_particles, n_dim))

sampler = pc.Sampler(n_particles,
                     n_dim, 
                     log_like_vec, 
                     log_prior, 
                     bounds=bound,
                     random_state=42
                     )

sampler.run(prior_samples)

sampler.add_samples(9000)

print("logZ estimated using PMC : ", np.round(sampler.results["logz"][-1],3))

Iter: 28it [01:56,  4.15s/it, beta=1, calls=57000, ESS=0.959, logZ=-5.77, accept=0.258, N=2, scale=1.27, corr=0.637]      
Iter: 9it [00:00, 10.60it/s, beta=1, calls=75000, ESS=0.95, logZ=-5.77, accept=0.238, N=2, scale=1.34, corr=0.656]

logZ estimated using PMC :  -5.768


## Bridge Sampling

Now that we have successfuly sampled the posterior, we can move on to estimate the log model evidence $\log\mathcal{Z}$ using Bridge Sampling.

In [3]:
logz_bs, logz_bs_error = sampler.bridge_sampling()

print("logZ estimated using Bridge Sampling : ", np.round(logz_bs,3), " +- ", np.round(logz_bs_error,3))

logZ estimated using Bridge Sampling :  -5.801  +-  0.003


The Bridge Sampling estimate is much closer to the true value of $\log\mathcal{Z}_{a}-5.804$ and the error-bar looks reasonable. Although the difference between the two methods is small in this example, this is not generally the case. In particular, for higher dimensional problems,  Bridge Sampling should always be preferred!